## Use this file to generate the slurm array configurations 
## Also, note that the code will generate the checkpoint folders in the folder that you specify below

In [1]:
# Remember to change the checkpoint directory
ckpt_root = '/data/user-name/Generalization_metrics_for_NLP/checkpoint/'

# Select the "sample x learning rate x depth" grid or the "sample x learning rate x width" grid
grid = 'depth'  # choices = ['depth', 'width']

# Select training or evaluating generalization metrics
experiment = 'train'  # choices = ['train', 'eval_metrics']

In [2]:
import os

assert os.path.exists(ckpt_root)
assert grid in ['depth', 'width']
assert experiment in ['train', 'eval_metrics']

lrs = [0.0625, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 1.0]
widths = [256, 384, 512, 768, 1024]
width_standard = 512
head_standard = 8
heads = [4, 6, 8, 12, 16]
samples = [160000, 320000, 640000, 1280000, 2560000]
depths = [4, 5, 6, 7, 8]
depth_standard = 6
dropout = 0.1

### Use the following code to generate the config files for training and evaluation

In [3]:
def change_configure_file(configs, sample, depth, width, lr, dropout, head):
    
    hyperparameter_string = f'WMT14_sample{sample}_depth{depth}_width{width}_lr{lr}_dropout{dropout}'
    ckpt_folder = os.path.join(ckpt_root, hyperparameter_string)
    if not os.path.exists(ckpt_folder):
        os.makedirs(ckpt_folder)
    
    for task, suffix in zip(tasks, suffixes):
        task_completion_file = os.path.join(ckpt_folder, suffix)

        if task == 'train':
            ## Check if the final training result exists.
            ## If not, put the task in the training list.
            if os.path.exists(task_completion_file):
                print(f"Task {task} finished for sample={sample}, lr={lr}, depth={depth}, width={width}.")
            else:
                configs[task].append(f"{sample} {depth} {width} {lr} {dropout} {head}")
        else:
            ## Check if the final training result exists and if the evaluation result does not exist.
            ## If so, put the task in the evaluation list.
            training_completion_file = os.path.join(ckpt_folder, 'net_epoch_20.ckpt')
            if os.path.exists(training_completion_file) and not os.path.exists(task_completion_file):
                configs[task].append(f"{sample} {depth} {width} {lr} {dropout} {head}")

                
if experiment=='train':
    tasks = ['train']
    suffixes = ['net_epoch_20.ckpt']
else:
    tasks = ['bleu', 'ww_tpl', 'ww_pl', 'ww_exponential', 'robust']
    suffixes = ['bleu_loss.jsonl', 'results.pkl', 'results_original_alpha.pkl', 'results_exponential.pkl', 'robust_measures.pkl']

configs = {x:[] for x in tasks}

## The following code only puts unfinished tasks to the configure file

for sample in samples:
    for lr in lrs:
        if grid == 'depth':
            for depth in depths:
                width = width_standard
                head = head_standard
                change_configure_file(configs, sample, depth, width, lr, dropout, head)
        else:
            for width, head in zip(widths, heads):
                depth = depth_standard
                change_configure_file(configs, sample, depth, width, lr, dropout, head)
                    
## write the unfinished tasks into the final configuration file
for task in tasks:
    with open(f'{task}_config.txt', 'w') as f:
        for line in configs[task]:
            f.write(line+'\n')